### import libraries and register pin 

In all examples you will need to import the following packages and register your pin in order to have access to the HydroData datasets

Refer to the [getting started](https://hydroframesubsettools.readthedocs.io/en/latest/getting_started.html) instructions for creating your pin if you have not done this already.

In [ ]:
import subsettools as st
import hf_hydrodata as hf
from parflow import Run

# register PIN / sign up here: https://hydrogen.princeton.edu/pin
# hf.register_api_pin("<email>", "<PIN>")

### get a template runscript

In [ ]:
# set static data directory and run name
# set a directory for the files to be written
# uncomment and set the path to your desired directory and include the HUC number
#static_write_dir = "/Users/<path>/<to>/<your>/<files>/huc0205"

#set the runname to be the HUC number you are working with
runname = "HUC0205"

template_run = st.get_template_runscript(
    grid="conus2", 
    mode="spinup", 
    input_file_type="solid",
    write_dir=static_write_dir,
)

### Modify a template runscript for a subset domain 
The runscript template is configured for a national simulation. To run on a smaller domain you need to: 
1. Adjust the size of the model to match your subset domain
2. Update the path to the model input files to match the clipped inputs for your subset domain (refer to the subset tutorials or sample workflow for examples of subsetting input files)

#### Modify the size of the model domain to math your subset
First we adjust the size of the model run (nx, ny) to match the dimensions of our subset area which we obtain using the i,j bounds function and provide the path to the subset forcing files. 

The `edit_runscript_for_subset` function (API reference [here](https://hydroframesubsettools.readthedocs.io/en/latest/autoapi/subsettools/index.html#subsettools.edit_runscript_for_subset)) will accomplish these two jobs. 

**NOTE:** *If you don't provide a write_dir argument, the new runscript is going to be written in the directory or the original runscript. The filename depends on the runname, so if you also don't change the runname the original runscript file will be overwritten.*

In [ ]:
# change the hucs=["0205"] to the desired HUC4
ij_huc_bounds, _ = st.define_huc_domain(hucs=["0205"], grid="conus2")
print(f"bounding box: {ij_huc_bounds}")


#Edit the runscript to match this subset
runscript_path = st.edit_runscript_for_subset(
    ij_bounds=ij_huc_bounds,
    runscript_path=template_run,
    runname=runname,
    forcing_dir=static_write_dir,
)

#Print out the keys of the new run to confirm the changes
run = Run.from_definition(runscript_path)
print('-' * 100)
print("New runname:", run.get_name())
print("New grid nx, ny:", run.ComputationalGrid.NX, run.ComputationalGrid.NY)
print("Forcing directory:", run.Solver.CLM.MetFilePath)

bounding box: (3672, 1827, 3963, 2223)
New runname: HUC0205 provided, a new yaml file will be created
Climate forcing directory has been changed to /Users/reed/Projects/headwaters_particles/huc0205  in runscript.
ComputationalGrid.NY set to 396 and NX to 291
GeomInput.domaininput.InputType detected as SolidFile, no additional keys to change for subset
Updated runscript written to /Users/reed/Projects/headwaters_particles/huc0205
----------------------------------------------------------------------------------------------------
New runname: HUC0205
New grid nx, ny: 291 396
Forcing directory: /Users/reed/Projects/headwaters_particles/huc0205


### Modify file paths for inputs in the runscript
We use the subsettools `change_filename_values` function  (API reference [here](https://hydroframesubsettools.readthedocs.io/en/latest/autoapi/subsettools/index.html#subsettools.change_filename_values)) to modify the filenames for the input files of our runscript. 

The filenames which can be updated from this function are: 
- Slope x (`slopex`)
- Slopey y (`slopey`)
- Soild File (`solidfile`)
- Initial Pressure(`init_press`)
- Subsurface Indicator (`indicator`)
- Depth to bedrock (`depth_to_bedrock`)  **note this is the flow barrier file not really depth to bedrock**
- Mannings Roughness (`mannings`)
- Evapotranspiration P-ET (`evap_trans`)


In [9]:
# change the file names for the initial pressure, mannings and slope_x, slope_y data in the runscript:
runscript_path = st.change_filename_values(    
    runscript_path=runscript_path,
    init_press='ss_pressure_head.pfb', 
    depth_to_bedrock='pf_flowbarrier.pfb',
    indicator='pf_indicator.pfb',
    mannings='mannings.pfb',
    slopex='slope_x.pfb',
    slopey='slope_y.pfb',
)

X Slopes filename changed to slope_x.pfb
Y Slopes filename changed to slope_y.pfb
Initial pressure filename changed to ss_pressure_head.pfb
Indicator filename changed to pf_indicator.pfb
Depth to bedrock filename changed to pf_flowbarrier.pfb
Mannings filename changed to mannings.pfb
Updated runscript written to /Users/reed/Projects/headwaters_particles/huc0205


In [10]:
# distribute input files 
runscript_path = st.dist_run(
    P=1,
    Q=1,
    runscript_path=runscript_path,
    working_dir=static_write_dir,
    dist_clim_forcing=False,
)

/tmp/ipykernel_52470/1852637705.py:2: DeprecationWarning: keyword argument 'P' is no longer valid. Use 'topo_p' instead.
  runscript_path = st.dist_run(
/tmp/ipykernel_52470/1852637705.py:2: DeprecationWarning: keyword argument 'Q' is no longer valid. Use 'topo_q' instead.
  runscript_path = st.dist_run(


No forcing directory provided, only distributing static inputs
Distributed ss_pressure_head.pfb with NZ 10
Distributed pf_flowbarrier.pfb with NZ 10
Distributed streams.pfb with NZ 1
Distributed pme.pfb with NZ 10
Distributed pf_indicator.pfb with NZ 10
Distributed slope_x.pfb with NZ 1
Distributed mask.pfb with NZ 1
Distributed slope_y.pfb with NZ 1
Distributed mannings.pfb with NZ 1
Distributed dem.pfb with NZ 1
